In [1]:
import sys
sys.path.append('/Users/celia/GitHub/mouse_bandit/data_preprocessing_code')
sys.path.append('/Users/celia/GitHub/mouse_bandit')
import support_functions as sf
import numpy as np
import pandas as pd
import scipy as sp
import bandit_preprocessing as bp
import sys
import os
import behavior_database

In [33]:
record_df = pd.read_csv('/Users/celia/GitHub/mouse_bandit/session_record.csv',index_col=0)
replace_row_shay = False # change to True if want to substitute old row with new version for imaging data
celia_session_record = True

### First time we ran this, had to add 'shay' to the old data base and add a new column for Markov
    record_df.insert(0,'Owner','shay')
    record_df.insert(5,'Markov', 0)

In [34]:
if record_df.columns[0] != 'Owner':
    record_df.insert(0,'Owner','shay') # add column for name of user
if record_df.columns[5] != 'Markov':
    record_df.insert(5,'Markov', 0) # add column for binary variable for Markov version of task. All previous values 0.

In [35]:
print(record_df.shape)
record_df.tail(5)

(801, 18)


,Owner,Session ID,Mouse ID,Date,Phase,Markov,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
796,shay,04072017_K13,K13,2017-04-07 00:00:00,2.0,0,0.9,0.1,50.0,50.0,459.0,7.0,372.0,0.88,2.0,1.5,35.0,35.0
797,shay,04072017_K7,K7,2017-04-07 00:00:00,2.0,0,0.8,0.2,50.0,50.0,449.0,6.0,319.0,0.77,2.0,1.5,35.0,35.0
798,shay,04072017_K9,K9,2017-04-07 00:00:00,2.0,0,0.8,0.2,50.0,50.0,299.0,3.0,179.0,0.63,2.0,1.5,35.0,35.0
799,shay,04072017_Q43,Q43,2017-04-07 00:00:00,2.0,0,0.8,0.2,50.0,50.0,555.0,8.0,406.0,0.87,2.0,1.5,35.0,35.0
800,shay,04072017_Q45,Q45,2017-04-07 00:00:00,2.0,0,0.8,0.2,50.0,50.0,583.0,7.0,394.0,0.78,2.0,1.5,35.0,35.0


## Add all days (multi mice) to celia_session_record

In [44]:
if celia_session_record == True:
    record_df = record_df.drop(record_df.index[:]) # if want to reset and start with (for new session record)
    
    MOUSE = ['Baby','Mustard','Raspberry','Sky','Wine', 'Dill']
    ROOT = '/Users/celia/Dropbox (HMS)/Celia_BehaviorFiles/'
    root_mouse={}
    for mouse in MOUSE: # make list of all mouse directories
        temp_dict = {mouse:os.path.join(ROOT,mouse)}
        root_mouse.update(temp_dict)

    date_folders = os.listdir(root_mouse[MOUSE[0]]) # assuming all mice have same dates 
    date_folders = date_folders[1:]
    count=0
    for mouse in MOUSE:
        for date in date_folders:
            curr_dir = os.path.join(root_mouse[mouse], date)
            record_df = behavior_database.add_session(curr_dir, record_df, 'celia')
            count+=1
    record_df.to_csv(path_or_buf='/Users/celia/GitHub/mouse_bandit/celia_session_record.csv')

In [45]:
count - 60

426

In [46]:
record_df.shape

(413, 18)

In [25]:
record_df[record_df['Date']=='2017-11-09']

,Owner,Session ID,Mouse ID,Date,Phase,Markov,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
123,celia,11092017_Mustard,Mustard,2017-11-09,2.0,1,0.7,0.3,NaN,NaN,649.0,17.0,401.0,0.76,2.0,1.0,35.0,35.0


In [10]:
29 + (70*5)

379

In [ ]:
record_df.tail(10)

## Add any of Shay's data or change existing rows to contain imaging column

In [ ]:
# any of Shay's old data to be loaded in (dates first)
DAY = '03102017'
ROOT = '/Users/celia/Dropbox (HMS)/Shay_BehaviorFiles/BehaviorData/datesfirst'
root_day = os.path.join(ROOT,DAY)

mouse_folders = os.listdir(root_day)

for mouse in mouse_folders:
    if not mouse[0] == '.':
        record_df = record_df[record_df['Session ID']!='%s_%s' % (DAY, mouse)] # remove the original row
        curr_dir = os.path.join(root_day,mouse)
        record_df = behavior_database.add_session(curr_dir,record_df,'shay', True)

if replace_row_shay == True:
    record_df.to_csv(path_or_buf='/Users/celia/GitHub/mouse_bandit/session_record.csv')

In [ ]:
record_df[record_df['Session ID']==session]

In [ ]:
# any of Shay's old data to be loaded in (mouse name first)
MOUSE = 'Q43'
ROOT = '/Users/celia/Dropbox (HMS)/Shay_BehaviorFiles/BehaviorData/'
root_mouse = os.path.join(ROOT,MOUSE)

date_folders = os.listdir(root_mouse)

for date in date_folders:
    if not date[0] == '.':
        record_df = record_df[record_df['Session ID']!='%s_%s' % (date, MOUSE)] # remove the original row
        curr_dir = os.path.join(root_mouse,date)
        record_df = behavior_database.add_session(curr_dir,record_df,'shay', True)
if replace_row_shay == True:
    record_df.to_csv(path_or_buf='/Users/celia/GitHub/mouse_bandit/session_record.csv')

## Troubleshooting with the add_session function

In [14]:
'''set the input parameters to test'''
imaging=False
root_dir = '/Users/celia/Dropbox (HMS)/celia_behaviorFiles/Mustard/11092017'
owner_id='celia'

In [15]:
"""set up dataframe"""

output_df = record_df.copy()

names = [
    'Owner',
    'Session ID',
    'Mouse ID',
    'Date',
    'Phase',
    'Markov',
    'Left Reward Prob',
    'Right Reward Prob',
    'Block Range Min',
    'Block Range Max',
    'No. Trials',
    'No. Blocks',
    'No. Rewards',
    'p(high Port)',
    'Decision Window Duration',
    'Min Inter-trial-interval',
    'Left Solenoid Duration',
    'Right Solenoid Duration',
]

    
'''
load in trial data
'''
if imaging==True:
    columns = ['Elapsed Time (s)','Since last trial (s)',
    'Trial Duration (s)','Port Poked','Right Reward Prob',
    'Left Reward Prob','Reward Given', 'center_frame', 'decision_frame']
else:
    columns = ['Elapsed Time (s)','Since last trial (s)',
    'Trial Duration (s)','Port Poked','Right Reward Prob',
    'Left Reward Prob','Reward Given']

In [16]:
'''load in file'''

for file in os.listdir(root_dir):
    if not file[0] == '.':
        file_name = os.path.join(root_dir,file)
                
        if 'trials.csv' in file:
            trials = pd.read_csv(file_name,names=columns)
            session_id = file[:file.index('_',9)]
                    
        elif 'parameters.csv' in file:
            params = pd.read_csv(file_name)

    #else:
        #file
        #raise ValueError('In a hidden folder')


In [ ]:
params

In [ ]:
os.listdir(root_dir)[0][0]

In [17]:
# calculate p(high port)
high_p_port = np.zeros(trials.shape[0])
        
for row in trials.iterrows():
    i = row[0]
    current_trial = row[1]
            
    if ((current_trial['Right Reward Prob'] > current_trial['Left Reward Prob']) & (current_trial['Port Poked'] == 1)):
        high_p_port[i] = 1
    elif ((current_trial['Right Reward Prob'] < current_trial['Left Reward Prob']) & (current_trial['Port Poked'] == 2)):
        high_p_port[i] = 1
  
 # determine what stage in training we are in        
if params['centerPokeTrigger'].values == 0:
    phase = 0
elif (params['leftRewardProb'].values == params['rightRewardProb'].values):
    phase = 1
else:
    phase = 2
           
# convert date to datetime object
date = session_id[:8]
date_str =  np.str(date)
if (len(date_str) == 7):
    date_str = '0' + date_str
datetime = pd.to_datetime(date_str,format='%m%d%Y')
        
# check whether ismarkov is already a variable stored in params
if hasattr(params, 'ismarkov')==False:
    params['ismarkov']=0 # if not, set ismarkov to zero
        
# correct for block range stats if markov task was run
if params.loc[0,('ismarkov')]==1:
    params.loc[:,('blockRangeMin', 'blockRangeMax')] ='NaN'

            
# create a dictionary with all information
record = {
    'Owner': owner_id,
    'Session ID': session_id,
    'Mouse ID': session_id[9:],
    'Date': datetime,
    'Phase': phase,
    'Markov': params['ismarkov'],
    'Left Reward Prob': params['leftRewardProb'].values,
    'Right Reward Prob': params['rightRewardProb'].values,
    'Block Range Min': params['blockRangeMin'],
    'Block Range Max': params['blockRangeMax'],
    'No. Trials': trials.shape[0],
    'No. Blocks': np.sum(np.diff(trials['Right Reward Prob'].values) != 0),
    'No. Rewards': np.sum(trials['Reward Given']),
    'p(high Port)': np.round(high_p_port.mean(),decimals=2),
    'Decision Window Duration': params['centerPokeRewardWindow'],
    'Min Inter-trial-interval': params['minInterTrialInterval'],
    'Left Solenoid Duration': params['rewardDurationLeft'],
    'Right Solenoid Duration': params['rewardDurationRight']
                 }
        
#create DataFrame         
output_df = output_df.append(pd.DataFrame(data=record,columns=names),ignore_index=True)  
output_df = output_df.drop_duplicates()   
    

In [18]:
output_df.tail(10)

,Owner,Session ID,Mouse ID,Date,Phase,Markov,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
302,celia,10232017_Wine,Wine,2017-10-23,2.0,1,0.7,0.3,NaN,NaN,820.0,12.0,522.0,0.84,2.0,1.0,40.0,40.0
303,celia,10242017_Wine,Wine,2017-10-24,2.0,1,0.3,0.7,NaN,NaN,1219.0,25.0,751.0,0.79,2.0,1.0,40.0,40.0
304,celia,10252017_Wine,Wine,2017-10-25,2.0,1,0.7,0.3,NaN,NaN,641.0,18.0,380.0,0.71,2.0,1.0,40.0,40.0
305,celia,10262017_Wine,Wine,2017-10-26,2.0,1,0.3,0.7,NaN,NaN,759.0,14.0,357.0,0.44,2.0,1.0,40.0,40.0
306,celia,10272017_Wine,Wine,2017-10-27,2.0,1,0.3,0.7,NaN,NaN,843.0,17.0,513.0,0.76,2.0,1.0,40.0,40.0
307,celia,10302017_Wine,Wine,2017-10-30,2.0,1,0.3,0.7,NaN,NaN,816.0,12.0,502.0,0.74,2.0,1.0,40.0,40.0
308,celia,10312017_Wine,Wine,2017-10-31,2.0,1,0.7,0.3,NaN,NaN,778.0,8.0,491.0,0.85,2.0,1.0,40.0,40.0
309,celia,11102017_Wine,Wine,2017-11-10,2.0,1,0.7,0.3,NaN,NaN,788.0,23.0,472.0,0.72,2.0,1.0,40.0,40.0
310,celia,11132017_Wine,Wine,2017-11-13,2.0,1,0.3,0.7,NaN,NaN,833.0,19.0,425.0,0.58,2.0,1.0,40.0,40.0
311,celia,11092017_Mustard,Mustard,2017-11-09,2.0,1,0.7,0.3,NaN,NaN,649.0,17.0,401.0,0.76,2.0,1.0,35.0,35.0
